In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

scala_version = "2.12"
spark_version = "3.5.3"

package= [f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
            'org.apache.kafka:kafka-clients:2.8.0' ]

spark = SparkSession.builder.master("local[*]").appName("kafka-example").config("spark.jars.packages", ",".join(package)).getOrCreate()

spark 

## Creating a Kafka Source for Batch Queries

In [6]:
#Create dataframe from Kafka data
topic_name = 'RandomNumber'
kafka_server = 'localhost:9092'
kafkaDf = spark.read.format("kafka").option("kafka.bootstrap.servers", kafka_server).option("subscribe", topic_name).option("startingOffsets", "earliest").load()

# format("kafka") chỉ định rằng bạn đang sử dụng Kafka làm nguồn dữ liệu.
# kafka.bootstrap.servers: Đây là tùy chọn bắt buộc để kết nối tới Kafka. Nó chứa danh sách các broker (máy chủ Kafka) để ứng dụng Spark có thể tìm và kết nối tới Kafka cluster.
#kafka_server được chỉ định là localhost:9092, nghĩa là Kafka broker đang chạy trên cổng 9092 của localhost.

#subscribe: Tùy chọn này chỉ định topic mà Spark sẽ đọc dữ liệu từ Kafka.
#startingOffsets: Tùy chọn này xác định điểm bắt đầu đọc dữ liệu từ Kafka.



In [20]:
#Show data (converting dataframe to pandas for cleaner view of data)
kafkaDf.toPandas()

,key,value,topic,partition,offset,timestamp,timestampType
0,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,0,2024-12-27 00:53:40.078,0
1,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,1,2024-12-27 00:53:45.079,0
2,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,2,2024-12-27 00:53:50.080,0
3,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,3,2024-12-27 00:53:55.080,0
4,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,4,2024-12-27 00:54:00.087,0
...,...,...,...,...,...,...,...
851,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,851,2024-12-27 23:56:07.107,0
852,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,852,2024-12-27 23:56:12.122,0
853,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,853,2024-12-27 23:56:17.130,0
854,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,854,2024-12-27 23:56:22.142,0


In [9]:
# Show streaming data using for loop


batchDF = kafkaDf.select(col('topic') #Chọn cột topic từ DataFrame Kafka, chứa tên của Kafka topic.
                        ,col('offset') #Chọn cột offset từ DataFrame Kafka, chứa offset của mỗi message trong Kafka topic.
                        ,col('value').cast('string').substr(12,1).alias('rand_number')) # Dữ liệu trong cột value được lưu dưới dạng byte và được chuyển thành chuỗi (string).
                                                                                        # Trích xuất ký tự thứ 12 từ giá trị chuỗi (dựa trên index 1-based) và chỉ lấy một ký tự. Điều này giả định rằng value chứa chuỗi có độ dài ít nhất 12 ký tự.
                                                                                        # ổi tên cột đã xử lý thành rand_number.
from time import sleep
from IPython.display import display, clear_output
for x in range(0, 2000):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Seconds passed: {x*5}")
        display(batchDF.toPandas()) #Chuyển đổi batchDF (Spark DataFrame) thành Pandas DataFrame để hiển thị dữ liệu. Điều này có thể dẫn đến giới hạn bộ nhớ nếu batchDF chứa nhiều dữ liệu.
        sleep(5)
        clear_output(wait=True) # Xóa kết quả hiển thị trước đó, đảm bảo màn hình luôn chỉ hiển thị dữ liệu mới nhất, tạo cảm giác như một live stream.
    except KeyboardInterrupt:
        print("break")
        break
print("Live view ended...")

Showing live view refreshed every 5 seconds
Seconds passed: 15


,topic,offset,rand_number
0,RandomNumber,0,0
1,RandomNumber,1,1
2,RandomNumber,2,2
3,RandomNumber,3,3
4,RandomNumber,4,4
...,...,...,...
1314,RandomNumber,1314,2
1315,RandomNumber,1315,2
1316,RandomNumber,1316,2
1317,RandomNumber,1317,2


break
Live view ended...


In [10]:
#Perform some data aggregation and show live results
batchCountDF = batchDF.groupBy('rand_number').count() # Gộp dữ liệu theo các giá trị duy nhất trong cột rand_number. Đếm số lần xuất hiện của mỗi giá trị duy nhất.
for x in range(0, 2000):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Seconds passed: {x*5}")
        display(batchCountDF.toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view ended...")

Showing live view refreshed every 5 seconds
Seconds passed: 5


,rand_number,count
0,7,11
1,3,46
2,8,11
3,0,1
4,5,11
5,6,11
6,9,11
7,1,111
8,4,11
9,2,111


break
Live view ended...


## Creating a Kafka Source for Streaming Queries

In [10]:
# Create Streaming dataframe from Kafka
streamRawDf = spark.readStream.format("kafka").option("kafka.bootstrap.servers", kafka_server).option("subscribe", topic_name).load()
streamDF = streamRawDf.select(col('topic'),col('offset'),col('value').cast('string').substr(12,1).alias('rand_number'))
checkEvenDF = streamDF.withColumn('Is_Even',col('rand_number').cast('int') % 2 == 0 ) # Tạo cột mới Is_Even để kiểm tra xem giá trị rand_number có chia hết cho 2 không.

In [12]:
#Write stream
from random import randint
randNum=str(randint(0,10000))
q1name = "queryNumber"+randNum
q2name = "queryCheckEven"+randNum
stream_writer1 = (streamDF.writeStream.queryName(q1name).trigger(processingTime="5 seconds").outputMode("append").format("memory"))
stream_writer2 = (checkEvenDF.writeStream.queryName(q2name).trigger(processingTime="5 seconds").outputMode("append").format("memory"))
query1 = stream_writer1.start()    
query2 = stream_writer2.start()

spark.sql(f"SELECT * FROM {q1name}").show()
spark.sql(f"SELECT * FROM {q2name}").show()



+-----+------+-----------+
|topic|offset|rand_number|
+-----+------+-----------+
+-----+------+-----------+

+-----+------+-----------+-------+
|topic|offset|rand_number|Is_Even|
+-----+------+-----------+-------+
+-----+------+-----------+-------+



In [14]:
#view results
for x in range(0, 2000):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Seconds passed: {x*5}")
        display(spark.sql(f"SELECT * FROM {q1name}").toPandas())
        display(spark.sql(f"SELECT * FROM {q2name}").toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break

Showing live view refreshed every 5 seconds
Seconds passed: 0


,topic,offset,rand_number


,topic,offset,rand_number,Is_Even


break
